# Assessment Problems

***
***

## Problem 1: Extending the Lady Tasting Tea

---

### Research and Development

---

#### Research Context

The [*Lady Tasting Tea*](https://en.wikipedia.org/wiki/Lady_tasting_tea) experiment, introduced by Sir Ronald A. Fisher in 1935, is a classic example of **design of experiments** and **hypothesis testing**.
A lady claimed she could distinguish whether milk was poured into a cup before or after the tea was added.

The original experiment used eight cups: four milk-first ('M') and four tea-first ('T').
The probability of perfectly identifying the milk-first cups by chance was very low (~0.0143).

Extending the experiment to twelve cups, with four milk-first and eight tea-first, allows us to **estimate the likelihood of success by chance in a larger experiment**.

---

#### Development Considerations

* **Representation:** Using 'M' and 'T' provides a clear and simple way to distinguish between milk-first and tea-first cups.  
* **Randomisation:** Each trial is independent, reflecting the null hypothesis.  
* **Simulation scale:** The experiment uses simple loop-based logic to maintain clarity and transparency in the implementation. 
* **Statistical insight:** Demonstrates probability estimation with explicit loops.  

***

### Experiment Setup

---

The process begins with the import of NumPy, which provides the foundation for randomisation and numerical computations throughout the experiment.

In [ ]:
import numpy as np

The experiment proceeds by defining the total number of cups, along with how many are milk-first and tea-first. A list is then created to represent the true arrangement of these cups.